In [1]:
from scipy.misc import imread,imresize
import numpy as np
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
import tensorflow as tf
label_emotions = ['anger','contempt','disgust','fear','happy','sadness','surprise']
label_genders = ['female','male']

In [2]:
server = 'localhost:9000'
host, port = server.split(':')
img = imread('test/husein.jpg')
img.shape

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


(423, 341, 3)

In [3]:
channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

In [4]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'mobilenet'
request.model_spec.signature_name = 'predict_classes'
request.inputs['image'].CopyFrom(
  tf.contrib.util.make_tensor_proto(img.astype(dtype=np.float32), shape=[423, 341, 3]))

In [5]:
result_future = stub.Predict(request, 30.)

In [18]:
print(label_genders[np.argmax(result_future.outputs['gender'].float_val)])
print(label_emotions[np.argmax(result_future.outputs['emotion'].float_val)])
print(result_future.outputs['age'].float_val[0]-9)

male
anger
22.0000057220459


In [19]:
img = imread('test/another-husein.jpg')
request.inputs['image'].CopyFrom(
  tf.contrib.util.make_tensor_proto(img.astype(dtype=np.float32), shape=img.shape))
result_future = stub.Predict(request, 30.)
print(label_genders[np.argmax(result_future.outputs['gender'].float_val)])
print(label_emotions[np.argmax(result_future.outputs['emotion'].float_val)])
print(result_future.outputs['age'].float_val[0]-9)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


male
surprise
22.0
